In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

In [ ]:
model_data = pd.read_parquet("data/model_data.parquet")

# Proportion d'articles avec des citations
Approche :
1. On modélise le comportement attendu des journaux, sous l'hypothèse qu'ils respectent la norme de représentativité
2. On isole le pouvoir explicatif de ce modèle pour mesurer le poids de cette norme
3. On étudie les résidus pour comprendre comment le comportement des journaux s'en écarte

On utilise pour cela les résidus relatifs moyens notés $RRM$, qui indiquent la direction et l’ampleur de l’erreur de prédiction en pourcentage de la valeur réelle.

$$\text{RRM} = \frac{1}{n} \sum_{i=1}^{n} \frac{\hat{y}_i - y_i}{y_i}$$

Un $RMM$ positif indique que le modèle surestime la part d’articles contenant des citations, c’est-à-dire qu’il surestime le respect de la norme de représentativité par le journal pour l’alignement politique donné ($\hat{y}_i ≥ y_i$). Cela signifie que le journal publie moins d’articles avec des citations de cet alignement politique que ce que la norme prévoirait. Pour simplifier la lecture, les valeurs sont inversées dans les tableaux : une valeur positive signale désormais une surreprésentation, et une valeur négative une sous-représentation.

*NB1 : Les données sont mensuelles.*
***NB2 : Les Verts ne sont pour l'instant pas représentés dans les données.***

## 1. Modèle de base
Les modèles estimés sont de la forme $Y_{i} = \beta_{i}T_i$ avec :
- $Y_{i}$ est la proportion des articles intégrant des citations de la nuance politique $i$ (avec $\sum_{i=1}^{n} Y_{i} = 1$)
- $T_i$ est la proportion des voix obtenue par les candidats de la nuance politique $i$ au premier tour des précédentes élections législatives

On estime donc...
- $\beta_{i}$ la pondération globale des résultats électoraux de la nuance politique $i$

L'hypothèse implicite est $\beta_{i} = 1 : les nuances politiques sont représentées identiquement à leur poids électoral une fois éliminée la couverture de l'action gouvernementale, avec un comportement identique des journaux.

In [ ]:
results = {}
pvalues = {}

for alignement in model_data["alignement_politique"].unique():
    subset = model_data[model_data["alignement_politique"] == alignement]

    if len(subset) >= 3:
        X = subset[["votes_share"]]
        y = subset["art_share"]

        model = sm.OLS(y, X).fit(cov_type='HC3')
        y_pred = model.predict(X)
        mean_relative_residual = ((y_pred - y) / y).mean()

        results[alignement] = {
            "r_squared": model.rsquared,
            "mean_residual": mean_relative_residual}
        pvalues[alignement] = model.pvalues.to_dict()

    else:
        results[alignement] = {
            "r_squared": None,
            "mean_residual": None}
        pvalues[alignement] = {
            "votes_share": None,
            "government": None}

In [ ]:
pval_records = []

for alignement, coeffs in pvalues.items():
    for coef_name, pval in coeffs.items():
        if pval is not None and pval > 0.001:
            pval_records.append({
                "alignement": alignement,
                "coefficient": coef_name,
                "pvalue": pval
            })

pval_summary = pd.DataFrame(pval_records)

r2_table = pd.DataFrame.from_dict(
    {alignement: results[alignement]['r_squared'] for alignement in results},
    orient='index',
    columns=['r_squared'])

bias_table = pd.DataFrame.from_dict(
    {alignement: results[alignement]['mean_residual'] for alignement in results},
    orient='index',
    columns=['mean_residual'])

In [ ]:
if len(pval_summary) == 0:
    print("No unsignificant coeffs (pval > 0.001)")
else:
    print(f"{len(pval_summary)} unsignificant coeffs (pval > 0.001)\n")
    print(pval_summary)

In [ ]:
(100*r2_table).style.format("{:.4f}")

In [ ]:
(-100*bias_table).style.format("{:.4f}")

## 2. Modèle de base + gouvernement
Les modèles estimés sont de la forme $Y_{i} = \alpha G_i + \beta_{i}T_i$ avec :
- $Y_{i}$ est la proportion des articles intégrant des citations de la nuance politique $i$ (avec $\sum_{i=1}^{n} Y_{i} = 1$)
- $G_i$ est une indicatrice valant 1 si le Premier ministre appartient à la nuance politique $i$
- $T_i$ est la proportion des voix obtenue par les candidats de la nuance politique $i$ au premier tour des précédentes élections législatives

On estime donc...
- $\alpha$ la prime que les journaux accordent à la nuance politique au pouvoir, sous la forme d'une fraction fixe des articles avec des citations
- $\beta_{i}$ la pondération globale des résultats électoraux de la nuance politique $i$

L'hypothèse implicite est $\beta_{i} = 1 - \alpha_j$ : les nuances politiques sont représentées identiquement à leur poids électoral une fois éliminée la couverture de l'action gouvernementale, avec un comportement identique des journaux.

In [ ]:
results = {}
pvalues = {}

for alignement in model_data["alignement_politique"].unique():
    subset = model_data[model_data["alignement_politique"] == alignement]

    if len(subset) >= 3:
        X = subset[["votes_share", "government"]]
        y = subset["art_share"]

        model = sm.OLS(y, X).fit(cov_type='HC3')
        y_pred = model.predict(X)
        mean_relative_residual = ((y_pred - y) / y).mean()

        results[alignement] = {
            "r_squared": model.rsquared,
            "mean_residual": mean_relative_residual}
        pvalues[alignement] = model.pvalues.to_dict()

    else:
        results[alignement] = {
            "r_squared": None,
            "mean_residual": None}
        pvalues[alignement] = {
            "votes_share": None,
            "government": None}

In [ ]:
pval_records = []

for alignement, coeffs in pvalues.items():
    for coef_name, pval in coeffs.items():
        if pval is not None and pval > 0.001:
            pval_records.append({
                "alignement": alignement,
                "coefficient": coef_name,
                "pvalue": pval
            })

pval_summary = pd.DataFrame(pval_records)

r2_table = pd.DataFrame.from_dict(
    {alignement: results[alignement]['r_squared'] for alignement in results},
    orient='index',
    columns=['r_squared'])

bias_table = pd.DataFrame.from_dict(
    {alignement: results[alignement]['mean_residual'] for alignement in results},
    orient='index',
    columns=['mean_residual'])

In [ ]:
if len(pval_summary) == 0:
    print("No unsignificant coeffs (pval > 0.001)")
else:
    print(f"{len(pval_summary)} unsignificant coeffs (pval > 0.001)\n")
    print(pval_summary)

In [ ]:
(100*r2_table).style.format("{:.4f}")

In [ ]:
(-100*bias_table).style.format("{:.4f}")

## 3. Modèle de base + gouvernement + journaux
Les modèles estimés sont de la forme $Y_{ij} = \alpha_jG_i + \beta_{ij}T_i$ avec :
- $Y_{ij}$ est la proportion des articles intégrant des citations de la nuance politique $i$ dans le journal $j$ (avec $\sum_{i=1}^{n} Y_{ij} = 1$)
- $G_i$ est une indicatrice valant 1 si le Premier ministre appartient à la nuance politique $i$
- $T_i$ est la proportion des voix obtenue par les candidats de la nuance politique $i$ au premier tour des précédentes élections législatives

On estime donc...
- $\alpha_j$ la prime que le journal $j$ accorde à la nuance politique au pouvoir, sous la forme d'une fraction fixe des articles avec des citations
- $\beta_{ij}$ la pondération des résultats électoraux de la nuance politique $i$ par le journal $j$

L'hypothèse implicite est $\beta_{ij} = 1 - \alpha_j$ : les nuances politiques sont représentées identiquement à leur poids électoral une fois éliminée la couverture de l'action gouvernementale.

In [ ]:
results = {}
pvalues = {}

for alignement in model_data["alignement_politique"].unique():
    results[alignement] = {}
    pvalues[alignement] = {}
    
    for journal in model_data["journal"].unique():
        subset = model_data[
            (model_data["alignement_politique"] == alignement) &
            (model_data["journal"] == journal)]
        
        if len(subset) >= 3:
            X = subset[["votes_share", "government"]]
            y = subset["art_share"]
            
            model = sm.OLS(y, X).fit(cov_type='HC3')
            y_pred = model.predict(X)
            mean_relative_residual = ((y_pred - y) / y).mean()
            
            results[alignement][journal] = {
                "r_squared": model.rsquared,
                "mean_residual": mean_relative_residual}
            
            pvalues[alignement][journal] = model.pvalues.to_dict()
            
        else:
            results[alignement][journal] = {
                "r_squared": None,
                "mean_residual": None}
            pvalues[alignement][journal] = {
                "votes_share": None,
                "government": None}

In [ ]:
pval_records = []

for alignement, journals in pvalues.items():
    for journal, coeffs in journals.items():
        for coef_name, pval in coeffs.items():
            if pval is not None and pval > 0.001:
                pval_records.append({
                    "alignement": alignement,
                    "journal": journal,
                    "coefficient": coef_name,
                    "pvalue": pval
                })

pval_summary = pd.DataFrame(pval_records)

r2_table = pd.DataFrame({
    alignement: {journal: results[alignement][journal]['r_squared'] for journal in results[alignement]}
    for alignement in results}).T

bias_table = pd.DataFrame({
    alignement: {journal: results[alignement][journal]['mean_residual'] for journal in results[alignement]}
    for alignement in results}).T

In [ ]:
if len(pval_summary) == 0:
    print("No unsignificant coeffs (pval > 0.001)")
else:
    print(f"{len(pval_summary)} unsignificant coeffs (pval > 0.001)\n")
    print(pval_summary)

Almost all coefficients for the share of votes at legislatives elections and presence in the governement are highly significant.

In [ ]:
#r2_table.style.format("{:.4f}")

plt.figure(figsize=(10, 6))
sns.heatmap(100*r2_table, annot=True, fmt=".1f", cmap="RdBu_r", center=0, cbar_kws={'label': 'R2 (%)'})
plt.title("R2 by Journal and Political Alignment")
plt.ylabel("Political Alignment")
plt.xlabel("Journal")
plt.tight_layout()
plt.show()

All journals except Médiapart closely adhere to the representativity norm for non-extreme political alignments, with $R^2$ values exceeding 90%. Médiapart follows this norm to a lesser extent, with $R^2$ values ranging from 64% to 83.5%.

For the far right, all journals reflect its vote share to a similar degree, with $R^2$ values between 65% and 77% — highest for Le Figaro, lowest for La Croix.

The representativity norm appears slightly less consistent for the far left, with $R^2$ values ranging from 53% to 77%, again highest for Le Figaro and lowest for La Croix.

Possible explanations: the number of articles with quotes is driven by...
- The number of seats in the Assemblée Nationale, which tends to better reflect the scores of dominant political parties.
- The anticipated scores at the presidential élection.

Of note, the time periods covered vary across journals! And the Greens are currently not included in the dataset.

*NB: not shown here is that the government control has negligible effect.*

In [ ]:
# (-100*bias_table).style.format("{:.1f}")

plt.figure(figsize=(10, 6))
sns.heatmap(-100*bias_table, annot=True, fmt=".1f", cmap="RdBu_r", center=0, cbar_kws={'label': 'MRR (%)'})
plt.title("Mean Relative Residuals by Journal and Political Alignment")
plt.ylabel("Political Alignment")
plt.xlabel("Journal")
plt.tight_layout()
plt.show()

- *Le Figaro* and *Le Monde* display similar patterns: they broadly align with the norm for the conventional right, slightly favor the conventional left, slightly penalize the extremes, and strongly favor the center.
- *La Croix* stands out for imposing the strongest penalty on the far right—the most pronounced underrepresentation among all journals.
- *Libération* notably penalizes both the center and the far right.
- *Médiapart* underrepresents the conventional and far right, while favoring the far left.

*NB: Government control has a notable moderating effect here—halving the MRR for the center, leaving the right mostly unchanged, and slightly increasing it for the left.*

In [ ]:
## 4. Modèle de base + gouvernement + journaux + périodes